# Profitable App Profiles for the App Store and Google Play Markets

* What the project is about
* The goal of this project
our goal is to determine the kinds of apps that are likely to attract more users because the number of people using our apps affect our revenue.

Filter out apps that are NOT free to download and install.
Filter out non-English apps.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row) 
        print('\n') # adds a new (empty) line after each row
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

---
**In the code below I open the two datasets that I'm going to work with:**
* AppleStore.csv
* googleplaystore.csv
---
1. I open both datasets with the open() function.
2. To read the datasets I use the reader() function from the csv module.
3. Then I convert them into lists, so I can work with them.


In [3]:
from csv import reader

opened_file = open('AppleStore.csv', encoding='utf8')
apple_store = list(reader(opened_file))
ios_header = apple_store[0]
ios = apple_store[1:]

opened_file = open('googleplaystore.csv', encoding='utf8')
google_store = list(reader(opened_file))
android_header = google_store[0]
android = google_store[1:]

---
**Deleting incorrect data.**

Entry number 10472 is missing the 'Rating' and its next columns have shifted.

In [4]:
print(android_header)
print("\n")
print(android[10472])



['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


---
**Deleting entry number 10472.**

In [5]:
del android[10472]

---
**Now I start the data cleaning process. As can be seen below the number of duplicate apps is over 1000.**

In [6]:
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name not in unique_apps:
        unique_apps.append(name)
    else:
        duplicate_apps.append(name)

print("Number of duplicate apps:", len(duplicate_apps))
print("Examples of duplicate apps:", duplicate_apps[:11])

Number of duplicate apps: 1181
Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic']


---
I am going to remove duplicate entries, but NOT randomly.
There are 4 entries for the app 'Instagram', but only one variable changes between these entries, the 'Reviews'.
I am going to keep the entry with the highest number of revies because that should be the most recent data.

In [7]:
print(android_header)
print("\n")
for app in android:
    name = app[0]
    if name == 'Instagram':
        print(app)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


---
Creating a dictionary where each key is a unique app name and the corresponding dictionary value is the highest number of reviews of that app.

In [8]:
reviews_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

print("Number of entries:", len(reviews_max))
print(reviews_max["Instagram"])
        

Number of entries: 9659
66577446.0


---
Using the dictionary I created earlier to remove the duplicate entries.
The new and cleaned datasets will be stored in the 'android_clean' list.
I don't need to do the same for the App Store data because there are no duplicates.

In [9]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)

---
**Identifying Non-English Apps.**

Each character we use in a string has a corresponding number associated with it. 

The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the ASCII (American Standard Code for Information Interchange) system.

If the number is equal to or less than 127, then the character belongs to the set of common English characters.

If an app name contains a character that is greater than 127, then it probably means that the app has a non-English name.

To minimize the impact of data loss, I'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range. This means all English apps with up to three emoji or other special characters will still be labeled as English.

* If the is_english function returns True it's an English app, if it returns False the app is NOT.

In [10]:
def is_english(text):
    count = 0
    for letter in text:
        if ord(letter) > 127:
            count += 1
    if count > 3:
        return False
    else:
        return True
    
print(is_english('Instagram'))   
print(is_english('爱奇艺PPS爱奇'))

True
False


---
I am using the new function I created earlier (is_english()) to filter out non-English apps from both datasets, Applestore & googleplaystore.

In [11]:
ios_english = []
android_english = []

for app in ios:
    name = app[1]
    if is_english(name):
        ios_english.append(app)

for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app)
        
print("Number of english apps in AppleStore:", len(ios_english))
print("Number of english apps in Google Play Store:", len(android_english))

Number of english apps in AppleStore: 6183
Number of english apps in Google Play Store: 9614


**For the last step of the data cleaning process I will isolate the free apps from both datasets. (free of inaccurate data, free of duplicate entries & free of non-English apps)**

In [12]:
ios_free = []
android_free = []

for app in ios_english:
    price = float(app[4])
    if price == 0.0:
        ios_free.append(app)
        
for app in android_english:
    if app[7][0] == '$':
        price = float(app[7][1:])
    else:
        price = float(app[7])
    if price == 0.0:
        android_free.append(app)

print("Number of free english apps in AppleStore:", len(ios_free))
print("Number of free english apps in Google Play Store:", len(android_free))

Number of free english apps in AppleStore: 3222
Number of free english apps in Google Play Store: 8864


## Analysis of the most common genres for each market.
For this I will build 2 functions that I can use to analyze the frequency tables:
* One function to generate frequency tables that show percentages.
* Another function to diplay percentages in a descending order.

In [30]:
def freq_table(dataset, index):
    old_category = []
    category_dic = {}
    counter = 0
    for x in dataset:
        if x[index] not in old_category:
            old_category.append(x[index])
            category_dic[x[index]] = 1
        elif x[index] in old_category:
            category_dic[x[index]] += 1
        
        counter += 1
        
    for x in category_dic:
        category_dic[x] = round((category_dic[x] / counter * 100), 2)
    
    return category_dic

            

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0], '%')

print('App genres of free english apps in AppStore: ', '\n')
display_table(ios_free, 11)
print('\n')
print('App categories of free english apps in Google Play Store: ', '\n')
display_table(android_free, 1)
print('\n')
print('App genres of free english apps in Google Play Store: ', '\n')
display_table(android_free, 9)

App genres of free english apps in AppStore:  

Games : 58.16 %
Entertainment : 7.88 %
Photo & Video : 4.97 %
Education : 3.66 %
Social Networking : 3.29 %
Shopping : 2.61 %
Utilities : 2.51 %
Sports : 2.14 %
Music : 2.05 %
Health & Fitness : 2.02 %
Productivity : 1.74 %
Lifestyle : 1.58 %
News : 1.33 %
Travel : 1.24 %
Finance : 1.12 %
Weather : 0.87 %
Food & Drink : 0.81 %
Reference : 0.56 %
Business : 0.53 %
Book : 0.43 %
Navigation : 0.19 %
Medical : 0.19 %
Catalogs : 0.12 %


App categories of free english apps in Google Play Store:  

FAMILY : 18.91 %
GAME : 9.72 %
TOOLS : 8.46 %
BUSINESS : 4.59 %
LIFESTYLE : 3.9 %
PRODUCTIVITY : 3.89 %
FINANCE : 3.7 %
MEDICAL : 3.53 %
SPORTS : 3.4 %
PERSONALIZATION : 3.32 %
COMMUNICATION : 3.24 %
HEALTH_AND_FITNESS : 3.08 %
PHOTOGRAPHY : 2.94 %
NEWS_AND_MAGAZINES : 2.8 %
SOCIAL : 2.66 %
TRAVEL_AND_LOCAL : 2.34 %
SHOPPING : 2.25 %
BOOKS_AND_REFERENCE : 2.14 %
DATING : 1.86 %
VIDEO_PLAYERS : 1.79 %
MAPS_AND_NAVIGATION : 1.4 %
FOOD_AND_DRINK : 1.24 

## App profile recommendation for the App Store based on the number of user ratings.

In [38]:
genre_table = freq_table(ios_free, 11)

for genre in genre_table:
    total  = 0
    len_genre = 0
   
    for x in ios_free:
        genre_app = x[11]
        if genre_app == genre:
            total += int(x[5])
            len_genre += 1
    
    avg_rating = round(total / len_genre, 2)
    print(genre, '--', avg_rating)

Social Networking -- 71548.35
Photo & Video -- 28441.54
Games -- 22788.67
Music -- 57326.53
Reference -- 74942.11
Health & Fitness -- 23298.02
Weather -- 52279.89
Utilities -- 18684.46
Travel -- 28243.8
Shopping -- 26919.69
News -- 21248.02
Navigation -- 86090.33
Lifestyle -- 16485.76
Entertainment -- 14029.83
Food & Drink -- 33333.92
Sports -- 23008.9
Book -- 39758.5
Finance -- 31467.94
Education -- 7003.98
Productivity -- 21028.41
Business -- 7491.12
Catalogs -- 4004.0
Medical -- 612.0


## App profile recommendation for the Google Play Store based on the number of installs.

In [47]:
category_table = freq_table(android_free, 1)

for category in category_table:
    total = 0
    len_category = 0
    for x in android_free:
        category_app = x[1]
        if category_app == category:
            installs = x[5]
            installs = installs.replace(',', '')
            installs = installs.replace('+', '')
            total += int(installs)
            len_category += 1
            
    avg_installs = round(total / len_category, 2)
    print(category, '--', avg_installs)

ART_AND_DESIGN -- 1986335.09
AUTO_AND_VEHICLES -- 647317.82
BEAUTY -- 513151.89
BOOKS_AND_REFERENCE -- 8767811.89
BUSINESS -- 1712290.15
COMICS -- 817657.27
COMMUNICATION -- 38456119.17
DATING -- 854028.83
EDUCATION -- 1833495.15
ENTERTAINMENT -- 11640705.88
EVENTS -- 253542.22
FINANCE -- 1387692.48
FOOD_AND_DRINK -- 1924897.74
HEALTH_AND_FITNESS -- 4188821.99
HOUSE_AND_HOME -- 1331540.56
LIBRARIES_AND_DEMO -- 638503.73
LIFESTYLE -- 1437816.27
GAME -- 15588015.6
FAMILY -- 3695641.82
MEDICAL -- 120550.62
SOCIAL -- 23253652.13
SHOPPING -- 7036877.31
PHOTOGRAPHY -- 17840110.4
SPORTS -- 3638640.14
TRAVEL_AND_LOCAL -- 13984077.71
TOOLS -- 10801391.3
PERSONALIZATION -- 5201482.61
PRODUCTIVITY -- 16787331.34
PARENTING -- 542603.62
WEATHER -- 5074486.2
VIDEO_PLAYERS -- 24727872.45
NEWS_AND_MAGAZINES -- 9549178.47
MAPS_AND_NAVIGATION -- 4056941.77
